# Project: Sentences similarity

## Team members:

* Khanh Duong TRAN.
* Brandon NGUELEWI TOUTSAP.

## Goal: Compare two sentences similarity by calculating Cosine similarity
### Main ideas:

#### 1. Vectorize the sentences.
#### 2. Calculate Cosine similarity.
#### 3. Compare the similarity.
#### 4. Visualization between the 2 vectors.

### We will try to compare these two sentences:

 * "The weather is hot today!"
 * "It is a hot day."

#### Import libraries

In [105]:
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\brand\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\brand\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\brand\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\brand\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

### Vectorize sentences

#### Step includes:
##### 1. Process the sentences.
### comment: something
##### 2. Tokenize the sentences.
##### 3. Convert into array.

##### Process the sentences:
###### Step includes:

###### 1. Remove non-word character such as punctuation, numbers, emojis, etc.
###### 2. Remove redundant spaces.
###### 3. Convert all to lower case.

In [ ]:
def process_sentence(sentence: str):
    """ Preprocess the sentence before converting into array.

    Argument:
    sentence: The sentence we want to preprocess.

    Output: Clean sentence.
    """
    cleaned_sentence: str = ""
    only_w: str = ''.join(char for char in sentence if char.isalpha() or char.isspace())
    no_space: str = ' '.join(only_w.split())
    to_lwer: str = no_space.lower()
    cleaned_sentence = to_lwer

    return cleaned_sentence

In [ ]:
#Test the function
assert process_sentence("I haVe a nAme20") == "i have a name"

##### Tokenize the sentences:
###### Step includes:

###### 1. Split the sentences into each word individually.
###### 2. Lemmatize each word.
###### 3. Create a vocabulary to vectorize.

In [ ]:
#Credit: ChatGPT
def lemmatize_word(word):
    """ Return the word into it most basic form.

    Argument:
    word: Word that needs to be reduced into most basic form.
    """
    lemmatizer = WordNetLemmatizer()

    tag = nltk.pos_tag([word])[0][1][0].lower()
    tag_dict = {"a": wordnet.ADJ,
                "n": wordnet.NOUN,
                "v": wordnet.VERB,
                "r": wordnet.ADV}
    return lemmatizer.lemmatize(word, tag_dict.get(tag, wordnet.NOUN))

In [ ]:
def create_vocab(words_list):
    """ Return a dictionary of vocabulary for a list of words

    Argument:
    sentence: List of words that we want to create a dictionary of vocabulary.
    """
    lemmatized_list_of_word = [lemmatize_word(word) for word in words_list]

    unique_word_set: set = set(sorted(lemmatized_list_of_word))
    vocab: dict = {word: index for index, word in enumerate(unique_word_set)}

    return vocab

In [ ]:
vocab = create_vocab(["apple", "banana", "apple", "orange", "grape", "banana"])
vocab_2 = create_vocab(["studying", "watching", "was"])

assert 'apple' in vocab
assert 'orange' in vocab
assert 'pear' not in vocab

assert "study" in vocab_2
assert "watch" in vocab_2
assert "be" in vocab_2

##### Convert into array:
###### Base on the created vocabulary to convert a sentence into an array

In [ ]:
def vectorize(words_list, vocabulary):
    """ Return the vector of the sentence.

    Argument:
    words_list: list of word to make a vector.
    vocabulary: a dictionary of vocabulary from the sentences.
    """
    sentence_vector: np.ndarray = np.zeros(len(vocabulary))

    for word in words_list:
        if word in vocabulary:
            sentence_vector[vocabulary[word]] += 1

    return sentence_vector

In [ ]:
assert np.array_equal(vectorize(["apple", "banana", "apple", "orange", "grape", "banana"], {'apple': 0, 'banana': 1, 'grape': 2, 'orange': 3}), np.array([2, 2, 1, 1]))

#### Putting all together for vectorizing the sentences
##### This section summarizes all the previous steps before proceeding to the Cosine similarity calculation.

In [ ]:
def process_sentence(sentence):
    """ Preprocess the sentence and convert into an array of words.

    Argument:
    sentence: The sentence we want to preprocess.

    Output: Array of words.
    """

    #Remove non-word
    cleaned_sentence: str = ''.join(char for char in sentence if char.isalpha() or char.isspace())

    #Remove redundant whitespaces
    cleaned_sentence = ' '.join(cleaned_sentence.split())

    #Convert to lowercase
    cleaned_sentence = cleaned_sentence.lower()

    return cleaned_sentence.split()

def lemmatize_word(word):
    """ Return the word into it most basic form.

    Argument:
    word: Word that needs to be reduced into most basic form.
    """
    lemmatizer = WordNetLemmatizer()

    tag = nltk.pos_tag([word])[0][1][0].lower()
    tag_dict = {"a": wordnet.ADJ,
                "n": wordnet.NOUN,
                "v": wordnet.VERB,
                "r": wordnet.ADV}
    return lemmatizer.lemmatize(word, tag_dict.get(tag, wordnet.NOUN))

def create_vocab(sentences):
    """ Return a dictionary of vocabulary from two sentences

    Argument:
    sentences: List of sentences that we want to create a dictionary of vocabulary.
    """

    words = []
    for sentence in sentences:
        words_arr: np.ndarray = process_sentence(sentence)
        words += words_arr

    lemmatized_list_of_word = [lemmatize_word(word) for word in words]

    unique_word_set: set = set(sorted(lemmatized_list_of_word))
    vocab: dict = {word: index for index, word in enumerate(sorted(unique_word_set))} #using sorted() to make sure each run is consistent.

    return vocab

def vectorize(sentence, vocabulary):
    """ Return the vector of the sentence.

    Argument:
    sentence: The sentence we want to vectorize.
    vocabulary: A dictionary of vocabulary from the predefined sentences.
    """
    sentence_vector: np.ndarray = np.zeros(len(vocabulary))

    words_list: np.ndarray = process_sentence(sentence)
    for word in words_list:
        if word in vocabulary:
            sentence_vector[vocabulary[word]] += 1

    return sentence_vector

In [ ]:
sentence_1 = "Finally a sunny day!!"
sentence_2 = "It's a sunny day with clear skies."
sentences = np.array([sentence_1, sentence_2])

vocabulary = create_vocab(sentences)
print(vocabulary)

print(vectorize(sentence_1, vocabulary))
print(vectorize(sentence_2, vocabulary))

In [ ]:
vector_1 = vectorize(sentence_1, vocabulary)
vector_2 = vectorize(sentence_2, vocabulary)

## Calculate Cosine similarity

### The formula is:

$$
\cos(θ) =  \frac{A \cdot B}{\|A\| \times \|B\|}
$$

### Calculate the dot product of two vectors

In [ ]:
def calculate_dot_product(vector1, vector2):
    """ Return the dot product of two vectors.

    Argument:
    vector1 and vector2: two vectors.
    """
    dot_product: float = 0

    #Formula: dot = a1*b1 + a2*b2 ... + an*bn, with n is the size of the array.
    dot_product = sum(vector1[i] * vector2[i] for i in range(len(vector1)))

    return dot_product

In [ ]:
assert calculate_dot_product(vector_1, vector_2) == np.dot(vector_1, vector_2)

### Calculate the norm of the vectors

In [ ]:
def calculate_norm(vector):
    """ Return the norm of a vector.

    Argument:
    vector: Vector we want to calculate the norm.
    """

    #Formula: squared root of the sum squared of all elements in an array.
    sum_ele: int = sum(vector[i] ** 2 for i in range(len(vector)))
    return np.sqrt(sum_ele)

In [ ]:
assert calculate_norm(vector_2) == np.linalg.norm(vector_2)

### Finally, calculate the cosine similarity

In [ ]:
def cosine_similarity(vector1, vector2):
    """ Return the cosine similarity of two vectors.

    Argument:
    vector1 and vector2: vectors to calculate.
    """
    dot: int = calculate_dot_product(vector1, vector2)
    norm_multi: float = calculate_norm(vector1) * calculate_norm(vector2)

    cosine: float = dot / norm_multi
    cosine_percentage = cosine * 100
    return cosine_percentage

In [ ]:
similitude = cosine_similarity(vector_1, vector_2)
similitude

## Compare

### We define the threshold for the similarity is 0.5.

## Visualization

In [ ]:
def visualize_2_vectors(vector_1, vector_2, similitude):
    """ Visualize the two vectors onto a 2-D dimension space with an angle in between.

    Argument:
    vector_1 and vector_2: The two vectors we want to visualize.
    similitude: The calculated Cosine similarity
    """
    angle_rad = np.arccos(similitude/100)
    angle_deg = np.degrees(angle_rad)

    plt.figure(figsize=(6, 6))

    # Plot Vector 1 as an arrow
    plt.arrow(0, 0, vector_1[0], vector_1[1], color='blue', width=0.05, label='Vector 1', alpha=0.5)
    # Plot Vector 2 as an arrow
    plt.arrow(0, 0, vector_2[0], vector_2[1], color='orange', width=0.05, label='Vector 2', alpha=0.5)

    plt.xlim(-0.5, 1.5)
    plt.ylim(-0.5, 1.5)

    # Add the angle between vectors as text annotation
    plt.text(0.1, 1.2, f'Angle: {angle_deg:.2f}°', fontsize=12)

    plt.xlabel('Dimension 1')
    plt.ylabel('Dimension 2')
    plt.title('Vector Representation and Angle')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    plt.show()

In [ ]:
visualize_2_vectors(vector_1, vector_2, similitude)

## An interactive way that can take user input to compare

#### We also made an interactive version so you can test this by yourself. Note that, since we coded this by hand, it would not be as robust as using libraries' tools.

#### Simply convert the below code cell into code by using `Esc` + `y`.

sen_1: str = input("Enter your first sentence: ")
sen_2: str = input("Enter your second sentence: ")
arr_sens: np.ndarray = np.array([sen_1, sen_2])

print(sen_1)
print(sen_2)

user_input_vocab: dict = create_vocab(arr_sens)

vec_1 = vectorize(sen_1, user_input_vocab)
vec_2 = vectorize(sen_2, user_input_vocab)

similarity: float = cosine_similarity(vec_1, vec_2)
print(similarity)

visualize_2_vectors(vec_1, vec_2, similarity)

## An addition to the project

### Now that we see how the cosine similarity works and how we can leverage it into determine whether a text is positive or negative.

### This is a subset of Data Science domain called **Sentiment Analysis**, frequently used to help improve Brands/products reputations on the market.

### Main idea:

#### 1. Pre-defined a positive and negative set of vocabulary to vectorize an unseen review.

#### 2. Calculate the cosine similarity between positive/negative and unseen review.

#### 3. Compare the two cosine, the higher *score* means it is positive or negative.

In [ ]:
#Taken some random reviews for vocabulary creation
positive_reviews = [
    "This product is amazing! It exceeded all my expectations.",
    "I love this app! It's so easy to use and has all the features I need.",
    "The service was excellent and the staff was very friendly.",
    "The food at this restaurant is delicious and the atmosphere is great.",
    "I had a wonderful experience with this company. Their customer support is top-notch.",
    "I'm impressed with the quality of this product. It's worth every penny.",
    "This book is a must-read! I couldn't put it down.",
    "The hotel staff went above and beyond to make our stay comfortable and enjoyable.",
    "I highly recommend this movie. It's entertaining and well-made.",
    "The concert was fantastic! The artist gave an incredible performance.",
    "This app is a lifesaver! It helped me stay organized and on track.",
    "The customer service team was so helpful and responsive.",
    "I'm so happy with my purchase. The product arrived quickly and in perfect condition.",
    "The staff at this store was friendly and knowledgeable.",
    "The hotel room was clean and spacious. I had a great night's sleep.",
    "I can't say enough good things about this restaurant. The food was outstanding.",
    "I am extremely satisfied with this service. It saved me so much time and effort.",
    "The movie was heartwarming and had a great message.",
    "I had a fantastic time at the amusement park. The rides were thrilling.",
    "This product is a game-changer. It's made my life so much easier.",
]

negative_reviews = [
    "This product is terrible. It broke after just a few days of use.",
    "I'm very disappointed with this app. It keeps crashing and has a lot of bugs.",
    "The service at this restaurant was awful. The staff was rude and unhelpful.",
    "I had a horrible experience with this company. They didn't deliver what they promised.",
    "The food at this place was terrible. It was cold and tasteless.",
    "This book was a waste of time. The plot was confusing and the characters were uninteresting.",
    "The hotel room was dirty and poorly maintained. I would not stay there again.",
    "I regret watching this movie. It was boring and poorly acted.",
    "The concert was a disaster. The sound quality was terrible and the artist seemed uninterested.",
    "I wouldn't recommend this product to anyone. It's not worth the money.",
    "This app is useless. It doesn't do what it's supposed to do.",
    "The customer service was a nightmare. I was put on hold for hours.",
    "I was really disappointed with the quality of the product. It fell apart after a few uses.",
    "The food at this restaurant was overpriced and tasted bland.",
    "The book was full of errors and typos. It was poorly edited.",
    "The hotel was noisy and I couldn't sleep well.",
    "The movie was a complete waste of time. The plot was predictable.",
    "I had a terrible experience with the delivery service. My package was lost.",
    "The amusement park was crowded and the lines were ridiculously long.",
    "This product is a scam. It doesn't work as advertised.",
]

# Create the review vocab to calculate and vectorize
reviews: np.ndarray = positive_reviews + negative_reviews
review_vocabulary: dict = create_vocab(reviews)

# Vectorize the positive and negative reviews individually
positive_vectors = [vectorize(review, review_vocabulary) for review in positive_reviews]
negative_vectors = [vectorize(review, review_vocabulary) for review in negative_reviews]

#Try different unseen review to test, taken from internet
#new_review: str = "I recently purchased this product and I have to say I'm impressed. It's exactly what I was looking for and it works perfectly. The customer service was also very helpful when I had a question. I highly recommend it!"
new_review: str = "I was really disappointed with this product. It didn't work as expected and the customer service was unhelpful. I would not recommend it to others."

# Vectorize the new review
review_vector = vectorize(new_review, review_vocabulary)

# Calculate cosine similarity for the new review with each set of positive and negative reviews
cosine_positive = [cosine_similarity(review_vector, vector) for vector in positive_vectors]
cosine_negative = [cosine_similarity(review_vector, vector) for vector in negative_vectors]

# Determine sentiment based on cosine similarity
print(max(cosine_positive))
print(max(cosine_negative))

if max(cosine_positive) > max(cosine_negative):
    print("Positive review")
else:
    print("Negative review")